In [1]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [3]:
df=pd.read_stata("D://stata//BWdisclosure audits//2024 June cleaning analysis 7 countries//240619.dta")
df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
CPCleanedUnique=list(df.CPCleaned.unique())

C:\Users\Chunyun Li\AppData\Local\Temp\ipykernel_23720\3448851165.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())


In [6]:
df.head(3)

,index,Country,FactoryAssessedID,AssesmentDate,QuestID,CP,Q,Finding,Tags,disclosed,CPCleaned,QCleaned,id,year
0,611407,Cambodia,1029,06/06/2016,15496,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,611407,2016
1,589245,Vietnam,2151,15/12/2020,40869,overtime wages,does the employer pay workers correctly for al...,0,,0.0,overtime wages,correct pay for overtime at weekly rest days,589245,2020
2,610455,Cambodia,807,18/05/2021,42608,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,610455,2021


In [8]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

# def groupSent(sentences, threshold=0.75):
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     embeddings = model.encode(sentences,show_progress_bar=False)
#     clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
#     clustering.fit(embeddings)
#     clusters = {}
#     for sentence_id, cluster_id in enumerate(clustering.labels_):
#         if cluster_id not in clusters:
#             clusters[cluster_id] = []
#         clusters[cluster_id].append(sentences[sentence_id])
#     return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

In [11]:
tab(cp='osh management systems')

osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer develop an osh plan annually?,0,0,0,0,0,0,1802,1802
does the employer have an osh license?,0,0,0,0,0,119,0,119
does the employer have an steam generator license?,0,0,0,0,0,122,0,122
does the employer have osh technical organizational regulations (tor) that are approved by the ministry of labor?,0,0,0,0,0,119,0,119
does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?,0,0,0,833,0,0,0,833
does the employer inspect and measure the environmental conditions in the workplace on a annual basis?,0,0,0,0,0,0,1802,1802
does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?,0,0,0,0,0,122,0,122
does the employer record work-related accidents?,584,2499,161,851,474,119,1802,6490
"does the employer regularly inspect and maintain machines, equipment, buildings and stores?",0,0,0,0,0,0,1802,1802


In [ ]:
replace(["a","b","c"],"d")

In [12]:
getOriginalQ("mechanisms/committee for cooperation on osh")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?                                           2558
has the employer set up a properly functioning unit in charge of osh and/or osh council?                                                             1252
does the factory have a functioning safety committee?                                                                                                 584
has the employer set up a properly functioning unit in charge of osh and/or labour protection council?                                                550
does the factory have an osh committee ?                                                                                                              498
has the employer formed a joint worker/management osh committee?                                                                                      474
does the factory have an osh committee?                                   

In [13]:
collapse("mechanisms/committee for cooperation on osh","Cambodia")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2503
Name: count, dtype: int64

In [ ]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the workplace have a functioning hiv/aids committee?"

In [ ]:
df.loc[df.Q.str.contains(""), "QCleaned"]=""

In [14]:
df.to_csv("audits620.csv", index=False)

# [convert](./convert.txt)

# grouping:
## machine grouping:

In [152]:
for i in CPCleanedUnique:
    print(df[df.CPCleaned==i].QCleaned.nunique(), "\t",i,sep="")

33	other grounds
32	worker protection
22	emergency preparedness
4	hazardous work and other worst forms
24	chemicals and hazardous substances
18	dialogue, discipline and disputes
28	paid leave
8	working environment
14	gender
13	minimum wages/piece rate wages
22	leave
37	social security and other benefits
5	collective bargaining
16	worker accommodation
8	method of payment
1	forced labour and overtime
8	interference and discrimination
11	coercion
25	termination
7	race and origin
11	religion and political opinion
2	freedom to associate
1	bonded labour
14	employment contracts
21	contracting procedures
16	regular hours
3	union operations
15	health services and first aid
17	welfare facilities
18	overtime
9	documentation and protection of young workers
4	premium pay
4	wage information, use and deduction
26	osh management systems
16	overtime wages
4	strikes
1	child labourers
4	prison labour


In [ ]:
for i in tqdm(CPCleanedUnique):
    if i not in ['union operations','freedom to associate','strikes','collective bargaining','interference and discrimination']:
        if df[df.CPCleaned==i].QCleaned.nunique()>1: # otherwise ml model stop due to error
            ruleList=groupSent(df[df.CPCleaned==i].QCleaned.unique())
            for j in ruleList:
                if sum([len(df[df.QCleaned==x]) for x in j])<=6476:
                    replace(j)
        # break

# manually grouping:

In [139]:
# undo("has the employer taken legally required measures to help workers with hiv/ aids-related illnesses?")

# undo("is sexual orientation a factor in decisions regarding pay?")

# undo("is an applicant's sexual orientation a factor in hiring decisions?")

# replace(['does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?',
# 'does the employer allow workers with hiv/aids-related illnesses to work for as long as medically fit in available, appropriate work?']
# )

# replace(['has the employer taken all legally required measures to reasonably help workers with aids-related illnesses?',
# 'has the employer taken legally required measures to help workers with hiv/ aids-related illnesses?'])

# replace([
#     'is age a factor in hiring, employment, or termination?',
#     "is an applicant's age a factor in hiring decisions?"
# ])

# replace([
#     "is a worker's real or perceived hiv/aids status a factor in decisions regarding termination or retirement?",
# "is a worker's real or perceived hiv/aids status a factor in the employer's decisions regarding termination or retirement?"
# ])

# replace(["have all accommodations required by national law been made for physically disabled persons?",
# "has the employer made required accommodations for physically disabled workers?"])

# replace(["does the employer hire one disabled worker for every 100 workers?	",
#          "does the employer comply with legal requirements regarding the hiring of disabled workers?"],
#          "compliance on hiring disabled workers")

In [143]:
tab(0)

other grounds


Country,Bangladesh,Cambodia,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,
are a worker's family responsibilities a factor in the employer's decisions regarding termination or retirement of workers?,0,0,0,0,0,27,27
are disabled persons who apply for work evaluated according to their ability to perform the job?,0,2237,0,0,119,1802,4158
are family responsibilities a factor in decisions regarding opportunities for promotion or access to training?,0,0,0,0,0,27,27
are family responsibilities a factor in decisions regarding pay?,0,0,0,0,0,54,54
are hiv/aids tests required at hiring?,0,0,0,0,0,1802,1802
are workers harassed on the basis of their family responsibilities?,0,0,0,0,0,27,27
compliance on hiring disabled workers,0,3,851,474,0,0,1328
do recruitment materials such as job announcements or job application forms refer to age in a manner that could discourage older workers from applying?,0,0,0,0,0,721,721
"does the employer allow workers with aids-related illnesses to work for as long as medically fit in available, appropriate work?",0,0,0,0,2,1802,1804


In [190]:
#1:

# replace(['are electrical wires, switches and plugs properly installed, grounded, and maintained?',
#          'are electrical wires, cables, switches, plugs and equipment (e.g. transformer, generator, electrical panel, circuit breakers) properly installed, grounded (for equipment), and maintained?'])

# replace(['are workers effectively trained and encouraged to use the personal protective equipment that is provided?	',
#          'are workers effectively trained and encouraged to properly use personal protective equipment and machines?'])

# replace(
#     ['are workers effectively trained to use the personal protective equipment and machines?',
# 'are workers effectively trained and encouraged to properly use personal protective equipment, machines and equipment?',
# 'are workers effectively trained and encouraged to properly use the personal protective equipment and machines?',
# 'are workers effectively trained and encouraged to use the personal protective equipment that is provided?',
# 'are workers effectively trained on occupational health and safety?',
# 'are workers effectively trained to use the personal protective equipment and machines?',
# 'are workers effectively trained to use the personal protective equipment that is provided?',
# 'are all workers trained and obligated to use machines and equipment safely?']
# )

# undo("are boilers registered or certified, as legally required?")

# replace(["do workers have suitable chairs?",
# "does the employer comply with ergonomic requirements?"])

# replace(["are electrical wires, cables, switches, plugs and equipment (e.g. transformer, generator, electrical panel, circuit breakers, electrical boilers) properly installed, grounded (for equipment), maintained, and certified?",
# "are electrical wires, cables, switches, plugs, and equipment (e.g. transformer, generator, electrical panel, circuit breakers) properly installed, grounded (for equipment), and maintained?",
# "are electrical wires, switches and plugs properly installed, grounded, and maintained?",
# "are electrical boxes, electrical wires, switches and plugs properly installed, grounded, and maintained?",
# "are electrical wires, switches and plugs properly installed, grounded, maintained and certified?",
# "are electrical wires, switches, plugs and appliances properly installed, grounded, and maintained?",
# "are electrical wires, switches, plugs and appliances properly installed, grounded, regularly checked and maintained?"])

# replace(["do the operators / technicians for boiler, generator, compressor, and pressure vessels have legally required permit and training?",
# "do steam generating machine operators have valid licenses?",
# "are boilers registered or certified, as legally required?",
# "do the operators/ technicians/ officers responsible for machinery/ equipment/ installations/ lifting equipment have the required license?",
# "do the technicians for electrical installations and boiler operators have the legally required license/ certification?"],
# "boiler or steam operator certified")

# replace(["are workers punished if they remove themselves from work situations that they believe present an imminent and serious danger to life or health?",
# "does the employer force workers to continue working when they have refused to work due to clear imminent and serious danger to their life or health?"],"prevent workers away from dangerous workplaces")

# replace(["does the employer provide workers with all necessary personal protective clothing and equipment?",
# "does the employer provide workers with all necessary personal protective clothing and equipment?"])

# undo("does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?")

# replace(["does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?",
# "does the employer comply with legal requirements related to work subject to strict occupational safety requirements?"])

# replace([
#     "does the employer comply with legal requirements related to machines and equipment subject to strict occupational safety requirements?",
# "does the employer comply with legal requirements related to machines, equipment subject to strict occupational safety requirements?"
# ])

# replace(['does the employer provide  workers with all necessary personal protective clothing and equipment?',
#        'does the employer provide workers with all necessary personal protective clothing and equipment?'])

# replace(["does the employer provide workers with all necessary personal protective clothing and equipment?",
# "has the employer provided workers who work with chemical substances with proper clothing and personal protective equipment when necessary?"])

# replace([
#     "boiler or steam operator certified",
#     "do workers use the personal protective equipment that is provided?"
# ])

# for i in ["has the employer provided workers with dust masks when necessary?",
# "has the employer provided workers with ear plugs when necessary?",
# "has the employer provided workers with metal gloves when necessary?",
# "has the employer provided workers with necessary ppe to protect against wet processes when necessary?",
# "has the employer provided workers with other ppe when necessary?",
# "has the factory conducted an assessment of the potential hazards in the workplace?	"]:
#     assign(i)

# df.loc[df.Q=="do the technicians for electrical installations and boiler operators have the legally required license/ certification?","QCleaned"]=\
# "do the technicians for electrical installations and boiler operators have the legally required license/ certification?"


# for i in [
# "do workers use the machines and equipment in a safe manner?",
# "does the employer comply with legal requirements related to machines, equipment subject to strict occupational safety requirements?",
# "does the employer comply with legal requirements related to work subject to strict\noccupational safety requirements?",
# "has the factory conducted an assessment of the potential hazards in the workplace?",
# "is the workplace overcrowded?"
# ]:
#     assign(i)

In [191]:
# df[(df.QCleaned=="are electrical wires, switches and plugs properly installed, grounded, and maintained?") & (df.Country=="Indonesia")].Q.value_counts()
# df[(df.QCleaned=="boiler or steam operator certified") & (df.Country=="Bangladesh")].Q.value_counts()
tab(1)

worker protection


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are all workers trained and obliged to use the personal protective equipment that is provided?,0,0,0,834,0,0,0,834
are appropriate safety warnings posted in the workplace?,584,2503,161,833,474,119,1802,6476
"are electrical wires, switches and plugs properly installed, grounded, and maintained?",584,2503,161,838,474,122,1802,6484
are proper guards installed and maintained on all dangerous machines and equipment?,584,2503,161,833,474,119,1802,6476
are workers effectively trained on occupational health and safety?,584,2286,161,833,474,119,1802,6259
are workers protected against falls from heights?,357,2241,0,480,0,0,1802,4880
boiler or steam operator certified,357,2434,0,834,0,122,1802,5549
do the technicians for electrical installations and boiler operators have the legally required license/ certification?,357,0,0,0,0,0,0,357
do workers have suitable chairs?,584,2321,161,833,474,122,1802,6297


In [225]:
#2:

# replace(["does the workplace have adequate and accessible fire-fighting equipment?",
# "does the workplace have adequate fire-fighting equipment?"])

# replace(["does the workplace have adequate fire-fighting equipment and within easy reach of workers?",
# "does the workplace have adequate fire-fighting equipment that is within easy reach of workers?"])

# replace([
#     "does the workplace have adequate fire-fighting equipment and within easy reach of workers?",
# "does the workplace have adequate fire-fighting equipment?"
# ])

# replace([
#     "has the employer appointed and trained a fire management team?	",
#     "has the employer trained at least 80% of workers to use the fire-fighting equipment?"
# ])

# for i in [
#     "accessable unlocked emergency exits",
#     "are emergency exits and escape routes clearly marked?",
#           "at least 2 possible emergency exits","fire detection and alarm system","periodic emergency drills"]:
#     discardYear(i,"Nicaragua")

# for i in [
#     "accessable unlocked emergency exits",
#           "at least 2 possible emergency exits","fire detection and alarm system",
#           "does the workplace have adequate fire-fighting equipment?",
#           "has the employer appointed and trained a fire management team?",
#           "periodic emergency drills"]:
#     discardYear(i,"Indonesia")

replace([
    "has the employer appointed and trained a fire management team?",
    "has the employer trained an appropriate number of workers to use the fire-fighting equipment?"
])

In [226]:
tab(2)

emergency preparedness


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
accessable unlocked emergency exits,584,2503,161,833,474,119,1802,6476
are emergency exits and escape routes clearly marked?,584,2296,161,833,474,119,1802,6269
are escape routes free of obstruction?,0,2503,0,0,0,0,0,2503
are flammable materials safely stored?,584,2254,161,833,474,117,1802,6225
are floors and corridors in line with legal requirements?,0,0,0,0,0,117,0,117
are possible sources of ignition appropriately safeguarded?,584,2241,161,833,474,117,1802,6212
"are the emergency exits locked during working hours, including overtime?",0,2503,0,0,0,0,0,2503
at least 2 possible emergency exits,584,2503,161,833,474,119,1802,6476
does the employer comply with emergency requirements?,357,0,0,480,0,0,0,837


In [227]:
tab(3)

hazardous work and other worst forms


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are any workers under age 18 subjected to the unconditional worst forms of child labour?,584,2236,0,833,474,119,1802,6048
are any workers who are under age 18 working at night?,584,2236,161,833,474,119,1802,6209
are any workers who are under age 18 working overtime?,0,0,161,0,0,0,0,161
do workers under the age of 18 perform work that is hazardous by nature?,584,2236,161,833,0,119,1802,5735
All,1752,6708,483,2499,948,357,5406,18153


---


In [ ]:
#4:
discardYear("are chemicals and hazardous substances properly labeled?","Nicaragua")
discardYear("does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?","Nicaragua")
"has the employer effectively trained workers who work with chemicals and hazardous substances?"


In [229]:
tab(4)

chemicals and hazardous substances


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
are chemical mixing rooms properly equipped and used?,0,2238,0,0,0,0,0,2238
are chemicals and hazardous substances properly labeled?,584,2503,161,0,0,122,1802,5172
are chemicals and hazardous substances properly stored?,0,0,0,0,0,0,1802,1802
does the employer have chemical safety data sheets for all the hazardous chemicals used in the workplace?,0,0,161,0,0,0,0,161
"does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace, and are the chemicals properly labelled?",0,0,0,851,0,0,0,851
does the employer have chemical safety data sheets for the hazardous chemicals used in the workplace?,584,2279,0,0,0,122,0,2985
does the employer have the legally required license for storage and use of acids in the factory?,357,0,0,0,0,0,0,357
"does the employer keep a chemical inventory, and ensure that all chemicals and hazardous substances are properly labelled and have safety data sheets?",0,0,0,0,474,0,0,474
does the employer keep an inventory of chemicals and hazardous substances used in the workplace?,584,2293,161,833,0,119,0,3990


In [ ]:
df[(df.QCleaned=="storage of chemicals and hazardous substances")]

In [228]:
df.to_stata("240619.dta", version=118)